# Imports

In [3]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Detections

In [11]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Detect and render
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 230, 117), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [14]:
from IPython.display import Image, display

# Replace the URL with the actual image URL
image_url = "https://i.imgur.com/3j8BPdc.png"

# Display the image
display(Image(url=image_url, height=300))

# Determining Joints

In [15]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Detect and render
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 230, 117), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

c:\Users\salla\anaconda3\envs\pose\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[x: 0.599100173
y: 0.849819422
z: -2.30109358
visibility: 0.99755317
, x: 0.625659347
y: 0.761585355
z: -2.26714158
visibility: 0.997660637
, x: 0.645735204
y: 0.759108543
z: -2.26692343
visibility: 0.998096645
, x: 0.663217127
y: 0.755349576
z: -2.26738644
visibility: 0.997877717
, x: 0.561933458
y: 0.757014513
z: -2.26307464
visibility: 0.997180939
, x: 0.538600147
y: 0.753237128
z: -2.2636044
visibility: 0.996991634
, x: 0.514827967
y: 0.750579238
z: -2.26356506
visibility: 0.996170819
, x: 0.67729485
y: 0.752553225
z: -1.76021707
visibility: 0.998295367
, x: 0.480613172
y: 0.75754559
z: -1.72476482
visibility: 0.997258067
, x: 0.626851499
y: 0.903539538
z: -2.06818056
visibility: 0.980625629
, x: 0.56028229
y: 0.902267218
z: -2.06153512
visibility: 0.972387493
, x: 0.849788785
y: 0.925230563
z: -1.3320092
visibility: 0.865790725
, x: 0.404906213
y: 0.948667228
z: -1.21422887
visibility: 0.815071523
, x: 0.919937611
y: 1.14755511
z: -1.58334804
visibility: 0.360469401
, x: 0.3368594

In [16]:
len(landmarks)

33

In [20]:
for lm in mp_pose.PoseLandmark:
    print(lm, lm.name)

0 NOSE
1 LEFT_EYE_INNER
2 LEFT_EYE
3 LEFT_EYE_OUTER
4 RIGHT_EYE_INNER
5 RIGHT_EYE
6 RIGHT_EYE_OUTER
7 LEFT_EAR
8 RIGHT_EAR
9 MOUTH_LEFT
10 MOUTH_RIGHT
11 LEFT_SHOULDER
12 RIGHT_SHOULDER
13 LEFT_ELBOW
14 RIGHT_ELBOW
15 LEFT_WRIST
16 RIGHT_WRIST
17 LEFT_PINKY
18 RIGHT_PINKY
19 LEFT_INDEX
20 RIGHT_INDEX
21 LEFT_THUMB
22 RIGHT_THUMB
23 LEFT_HIP
24 RIGHT_HIP
25 LEFT_KNEE
26 RIGHT_KNEE
27 LEFT_ANKLE
28 RIGHT_ANKLE
29 LEFT_HEEL
30 RIGHT_HEEL
31 LEFT_FOOT_INDEX
32 RIGHT_FOOT_INDEX


In [21]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.863400459
y: 0.987034261
z: -0.495693594
visibility: 0.990209758

In [22]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.971828938
y: 1.46705413
z: -0.721810222
visibility: 0.0720495209

In [23]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.92629993
y: 1.78001094
z: -1.28272974
visibility: 0.050844457

# Calculating Angles

In [24]:
def calc_angle(a, b, c):
    a = np.array(a) #First
    b = np.array(b) #Mid
    c = np.array(c) #End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360.0-angle
    
    return angle

In [27]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [28]:
shoulder, elbow, wrist

([0.8634004592895508, 0.987034261226654],
 [0.9718289375305176, 1.4670541286468506],
 [0.9262999296188354, 1.7800109386444092])

In [29]:
calc_angle(shoulder, elbow, wrist)

np.float64(158.99412394878254)

In [36]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the image to RGB and process it with MediaPipe Pose
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # Convert back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            # Extract landmarks
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of shoulder, elbow, wrist
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angle
            angle = calc_angle(shoulder, elbow, wrist)

            # Visualize angle on the image
            cv2.putText(image, str(round(angle, 2)),
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error: {e}")

        # Render pose landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, 
                                  mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(
                                      color=(245, 117, 66), thickness=2, circle_radius=2),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(
                                      color=(245, 230, 117), thickness=2, circle_radius=2))

        # Display the image with annotations
        cv2.imshow('MediaPipe Feed', image)

        # Exit by pressing 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Curl Counter

In [39]:
cap = cv2.VideoCapture(0)

# Curl counter
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the image to RGB and process it with MediaPipe Pose
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # Convert back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            # Extract landmarks
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of shoulder, elbow, wrist
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angle
            angle = calc_angle(shoulder, elbow, wrist)

            # Visualize angle on the image
            cv2.putText(image, str(round(angle, 2)),
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Curl counter
            if angle>160:
                stage = "down"
            if angle<30 and stage == "down":
                stage = "up"
                counter+=1
                print(counter)

        except Exception as e:
            print(f"Error: {e}")

        # Render curl counter
        cv2.rectangle(image, (0,0), (225, 73), (245, 117, 16), -1)

        # Rep data
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (60,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # Render pose landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, 
                                  mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(
                                      color=(245, 117, 66), thickness=2, circle_radius=2),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(
                                      color=(245, 230, 117), thickness=2, circle_radius=2))

        # Display the image with annotations
        cv2.imshow('MediaPipe Feed', image)

        # Exit by pressing 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

c:\Users\salla\anaconda3\envs\pose\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1
2
3
4
